In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures,datetime
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

This parser works with just a few locations, constructing an ITA-style matrix of price vs start/end dates

In [45]:
# Setup relevant variables
ZIP = '60615'
AGENCYBR = 'ET'
AGENCYIDS = ['E11509', #oak park,
             'E11548', #bolingbrook
             'E115CH', #elmwood
             'E11516', #skokie
             'E11553', #Nn Lincolnwood 53
            ]
RADIUS = '30'

PICKUP = datetime.date(2017, 11, 5)
DROPOFF = PICKUP+datetime.timedelta(days=14)
FIXEDLENGTH = True
VARYSTART = 1
VARYEND = 2
TIME = '11:00 AM'

In [51]:
%run corefuncs_matrix_debug.ipynb

In [52]:
agencylist = getagencylist((PICKUP,DROPOFF))

In [53]:
print(agencylist)

{1: {'num': 1, 'type': 'city', 'brand': 'enterprise', 'code': 'E11586', 'name': 'Gg Hyde Park', 'bcode': 'ET', 'addr': '5508 South Lake Park', 'city': 'Chicago', 'state': 'IL', 'zip': '60637', 'dist': '0.95', 'lat': '41.794955', 'lon': '-87.587695'}, 2: {'num': 2, 'type': 'city', 'brand': 'enterprise', 'code': 'E11576', 'name': 'Gg South Loop', 'bcode': 'ET', 'addr': '2640 S Michigan Ave', 'city': 'Chicago', 'state': 'IL', 'zip': '60616', 'dist': '3.17', 'lat': '41.844252', 'lon': '-87.623689'}, 3: {'num': 3, 'type': 'city', 'brand': 'avis', 'code': 'CHIC05', 'name': 'Chicago - 18th Street & Wabash Avenue', 'bcode': 'AV', 'addr': '1830 South Wabash Avenue', 'city': 'Chicago', 'state': 'IL', 'zip': '60616', 'dist': '4.05', 'lat': '41.856873', 'lon': '-87.62607'}, 4: {'num': 4, 'type': 'city', 'brand': 'budget', 'code': 'CHIC02', 'name': 'Chicago', 'bcode': 'BG', 'addr': '1830 S Wabash Ave', 'city': 'Chicago', 'state': 'IL', 'zip': '60616', 'dist': '4.05', 'lat': '41.856873', 'lon': '-87

In [54]:
results = runparser(agencyids = AGENCYIDS, threads=5)

3
3
3
007 ND | 62 ('enterprise', 'E115CH', 'Dd Elmwood Park') ('14.15', '41.926269', '-87.821333')
3
3
013 ND | 68 ('enterprise', 'E11553', 'Nn Lincolnwood 53') ('16.04', '42.019014', '-87.709152')


In [19]:
print(results)

{9: {'num': 50, 'type': 'city', 'brand': 'enterprise', 'code': 'E11509', 'name': 'Dd Oak Park', 'bcode': 'ET', 'addr': '25 Chicago Ave', 'city': 'Oak Park', 'state': 'IL', 'zip': '60302', 'dist': '10.97', 'lat': '41.894663', 'lon': '-87.775929', 'start': '11/10/2017', 'end': '11/17/2017', 'Economy Car': 151.49, 'Compact Car': 151.53, 'Intermediate Car': 164.95, 'Standard Car': 207.22, 'Fullsize Car': 207.26, 'Premium Car': 351.78999999999996, 'Intermediate SUV': 279.52, 'Standard SUV': 387.91, 'Mini Van': 435.4, 'Fullsize SUV': 526.24, 'Luxury Car': 460.17, 'Premium SUV': 629.47, 'Fullsize Van': 1037.1799999999998}, 8: {'num': 50, 'type': 'city', 'brand': 'enterprise', 'code': 'E11509', 'name': 'Dd Oak Park', 'bcode': 'ET', 'addr': '25 Chicago Ave', 'city': 'Oak Park', 'state': 'IL', 'zip': '60302', 'dist': '10.97', 'lat': '41.894663', 'lon': '-87.775929', 'start': '11/09/2017', 'end': '11/16/2017', 'Economy Car': 146.93, 'Compact Car': 146.96, 'Intermediate Car': 184.73, 'Standard Car

In [55]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
rvals = np.array([x for x in results.values() if x is not None and desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:60]]

In [56]:
orderedresults = rvals[order]
print(desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:s} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['start']+'-'+k['end'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))

Compact Car Compact Car Intermediate Car
11/06/2017-11/20/2017 | 10.97 |   0 | 249.95 | 249.95 | 271.93 | enterprise | Dd Oak Park
11/04/2017-11/18/2017 | 17.54 |   0 | 252.22 | 252.22 | 274.40 | enterprise | Nn Skokie
11/04/2017-11/18/2017 | 16.04 |   0 | 252.22 | 252.22 | 274.40 | enterprise | Nn Lincolnwood 53
11/04/2017-11/18/2017 | 24.81 |   0 | 259.40 | 259.40 | 278.48 | enterprise | Aa Bolingbrook
11/04/2017-11/18/2017 | 10.97 |   0 | 259.94 | 259.94 | 281.92 | enterprise | Dd Oak Park
11/06/2017-11/20/2017 | 14.15 |   0 | 299.92 | 299.92 | 319.90 | enterprise | Dd Elmwood Park
11/06/2017-11/20/2017 | 17.54 |   0 | 302.62 | 302.62 | 322.78 | enterprise | Nn Skokie
11/06/2017-11/20/2017 | 16.04 |   0 | 302.62 | 302.62 | 322.78 | enterprise | Nn Lincolnwood 53
11/06/2017-11/20/2017 | 24.81 |   0 | 308.22 | 308.22 | 338.75 | enterprise | Aa Bolingbrook
11/05/2017-11/19/2017 | 10.97 |   0 | 319.88 | 319.88 | 329.87 | enterprise | Dd Oak Park
11/04/2017-11/18/2017 | 14.15 |   0 | 319